Ce notebook a pour objectif de tester quelques LLM:

1- **Llama-2-7B-AWQ:**	Llama 2(formé entre janvier 2023 et juillet 2023.) est une collection de modèles de texte génératifs pré-entraînés et affinés, dont l'échelle varie de 7 à 70 milliards de paramètres. Il s'agit du référentiel pour le modèle pré-entraîné 7B, converti pour le format Hugging Face Transformers.  
Le modèle prend en entrée du texte et retourne uniquement du texte.  
Llama 2 est un modèle linguistique auto-régressif qui utilise une architecture de transformateur optimisée. Les versions optimisées utilisent le supervised fine-tuning (SFT) et l'apprentissage par renforcement avec retour d'information humain (RLHF) pour s'aligner sur les préférences humaines en matière de serviabilité et de sécurité.

Noter que AWQ(Activation-aware Weight Quantization) est une méthode efficace, précise et extrêmement rapide de quantification par poids à faible bit, qui prend actuellement en charge la quantification à 4 bits. Par rapport à GPTQ, elle offre une inférence plus rapide basée sur les transformateurs. L'utilisation d'AWQ permet d'utiliser des GPU beaucoup plus petits, ce qui peut faciliter le déploiement et réduire les coûts globaux. Par exemple, un modèle 70B peut être exécuté sur 1 GPU de 48 Go au lieu de 2 GPU de 80 Go.

2- **Mistral:** Le Mistral-7B-v0.1 Large Language Model (LLM) est un modèle de texte génératif pré-entraîné avec 7 milliards de paramètres. Mistral-7B-v0.1 surpasse Llama 2 13B sur tous les points de référence testés(d'apres la page officielle: connaissance, raisonnement, compréhension, mathématiques,…). Mixtral est multilingue (anglais, français, italien, allemand, espagnol), se montrerait performant en programmation informatique et pourrait gérer une fenêtre de contexte (quantité de texte traiter en une fois) de 32’000 tokens (environ 25’000 mots). Mixtral a la particularité de posséder 46,7 milliards de paramètres au total, mais en n'utilisant que 12,9 milliards de paramètres par token. Car le modèle repose sur une approche dite «Mixture of Experts» (MOE) qui permet d'augmenter le nombre de paramètres d'un modèle tout en contrôlant le coût et la latence.

l’architecture MOE est composée de réseaux neuronaux distincts, nommés «experts», qui traitent chacun un sous-ensemble différent de données d'apprentissage. Une autre couche (un réseau de passerelles ou routeur) supervise et orchestre le distribution des tokens à ces différents experts, selon le poids de leur expertise pour une tâche donnée. A noter qu’il est possible de transmettre un token à plus d'un expert. Au cours de la formation du modèle, à la fois les experts et le système de délégation des tâches sont entraînés.

Selon les chercheurs de Mistral, les MOE présentent l’avantage de permettre un pré-entraînement nettement plus efficace en termes de ressources de calcul. En outre, l'inférence est beaucoup plus rapide qu'avec d’autres modèles comportant le même nombre de paramètres.

lien drive: https://drive.google.com/drive/folders/1gA4GrkLDmzHAUa7lUIFI2al4NZcLF1WP?usp=sharing

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/M2Datascale/Projet_Transversal/TestLLM'

/content/drive/MyDrive/M2Datascale/Projet_Transversal/TestLLM


In [ ]:
%pip install autoawq

In [ ]:
%pip install transformers ctransformers

In [2]:
import time
import pandas as pd

In [5]:
questions = pd.read_excel('questionsllm.xlsx')
data = questions.sample(n=10)

In [6]:
data

,Question,Answers
58,What are the symptoms of Parkinson's Disease?,Treatment includes medications (levodopa) and ...
117,How is HIV/AIDS diagnosed?,Treatment includes antiretroviral therapy (ART...
166,How can I manage my Dengue Fever?,Diagnosis involves blood tests to detect the v...
278,Can Glaucoma be prevented?,Preventing Glaucoma involves regular eye exams...
261,Can Endometriosis be prevented?,Endometriosis is caused by the abnormal growth...
205,How can I manage my Multiple Sclerosis?,"Parkinson's Disease symptoms include tremors, ..."
135,How is Cancer diagnosed?,Symptoms vary depending on the type and stage ...
260,What treatment options are available for Endom...,Managing Endometriosis involves pain managemen...
5,How can I manage my Diabetes?,"Medication adherence, blood sugar monitoring."
162,How is Dengue Fever diagnosed?,"Lifestyle changes include avoiding alcohol, pr..."


In [7]:
data.to_csv('data.csv')

### llama-index

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-7B-AWQ")
model = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7B-AWQ",trust_remote_code=True, device_map='auto')

In [10]:
def getLlamaResponse(question,reponse_attendue):
  prompt_template=f'''{question}

  '''
  tokens = tokenizer(
      prompt_template,
      return_tensors='pt'
  ).input_ids.cuda()

  # Generate output
  start_time=time.time()
  generation_output = model.generate(
      tokens,
      do_sample=True,
      temperature=0.7,
      top_p=0.95,
      top_k=40,
      max_new_tokens=512
  )
  response = tokenizer.decode(generation_output[0])
  duree = time.time() - start_time
  new_row={'Model':'Llama-2-7B-AWQ', 'Questions':question, 'Reponses':response, 'Temps':duree, 'Reponses_Attendues':reponse_attendue}
  return new_row

In [ ]:
list_data_llama=[]
for i in range(len(data)):
   list_data_llama.append(getLlamaResponse(data.iloc[i]['Question'],data.iloc[i]['Answers']))

In [12]:
df_llama = pd.DataFrame(data=list_data_llama)

In [13]:
df_llama

,Model,Questions,Reponses,Temps,Reponses_Attendues
0,Llama-2-7B-AWQ,What are the symptoms of Parkinson's Disease?,<s> What are the symptoms of Parkinson's Disea...,28.271754,Treatment includes medications (levodopa) and ...
1,Llama-2-7B-AWQ,How is HIV/AIDS diagnosed?,<s> How is HIV/AIDS diagnosed?\n\n \n\n## Tre...,25.268855,Treatment includes antiretroviral therapy (ART...
2,Llama-2-7B-AWQ,How can I manage my Dengue Fever?,<s> How can I manage my Dengue Fever?\n\n 1. ...,25.007585,Diagnosis involves blood tests to detect the v...
3,Llama-2-7B-AWQ,Can Glaucoma be prevented?,<s> Can Glaucoma be prevented?\n\n The answe...,25.540988,Preventing Glaucoma involves regular eye exams...
4,Llama-2-7B-AWQ,Can Endometriosis be prevented?,<s> Can Endometriosis be prevented?\n\n \n\n\...,24.276372,Endometriosis is caused by the abnormal growth...
5,Llama-2-7B-AWQ,How can I manage my Multiple Sclerosis?,<s> How can I manage my Multiple Sclerosis?\n\...,25.282491,"Parkinson's Disease symptoms include tremors, ..."
6,Llama-2-7B-AWQ,How is Cancer diagnosed?,<s> How is Cancer diagnosed?\n\n \n * \n ...,25.168608,Symptoms vary depending on the type and stage ...
7,Llama-2-7B-AWQ,What treatment options are available for Endom...,<s> What treatment options are available for E...,21.867946,Managing Endometriosis involves pain managemen...
8,Llama-2-7B-AWQ,How can I manage my Diabetes?,<s> How can I manage my Diabetes?\n\n \n\n## ...,25.357483,"Medication adherence, blood sugar monitoring."
9,Llama-2-7B-AWQ,How is Dengue Fever diagnosed?,<s> How is Dengue Fever diagnosed?\n\n \n\nDe...,8.786251,"Lifestyle changes include avoiding alcohol, pr..."


# Mistral

In [ ]:
from ctransformers import AutoModelForCausalLM
# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-v0.1-GGUF", 
                                           model_file="mistral-7b-v0.1.Q4_K_M.gguf",
                                           model_type="mistral", gpu_layers=50)

In [15]:
def getMistralResponse(question,reponse_attendue):
  start_time = time.time()
  response = llm(question)
  duree = (time.time() - start_time)
  new_row={'Model':'Mistral-7B', 'Questions':question, 'Reponses':response, 'Temps':duree, 'Reponses_Attendues':reponse_attendue}
  return new_row

In [16]:
list_data_mistral=[]
for i in range(len(data)):
   list_data_mistral.append(getMistralResponse(data.iloc[i]['Question'],data.iloc[i]['Answers']))

In [17]:
df_mistral = pd.DataFrame(data=list_data_mistral)

In [18]:
df_mistral

,Model,Questions,Reponses,Temps,Reponses_Attendues
0,Mistral-7B,What are the symptoms of Parkinson's Disease?,\n\nThe most common symptoms of Parkinson’s di...,14.755228,Treatment includes medications (levodopa) and ...
1,Mistral-7B,How is HIV/AIDS diagnosed?,\nThe virus is present in blood and can be det...,12.599415,Treatment includes antiretroviral therapy (ART...
2,Mistral-7B,How can I manage my Dengue Fever?,What homeopathy treatments work to combat mos...,13.020158,Diagnosis involves blood tests to detect the v...
3,Mistral-7B,Can Glaucoma be prevented?,"\n\nYes, if it is detected early and treated p...",12.742887,Preventing Glaucoma involves regular eye exams...
4,Mistral-7B,Can Endometriosis be prevented?,\n\nCan you prevent endometriosis? Yes and No....,8.878620,Endometriosis is caused by the abnormal growth...
5,Mistral-7B,How can I manage my Multiple Sclerosis?,"\n\nI’m a newcomer to MS, only being diagnosed...",12.499902,"Parkinson's Disease symptoms include tremors, ..."
6,Mistral-7B,How is Cancer diagnosed?,\nA combination of several tests are used to d...,12.056664,Symptoms vary depending on the type and stage ...
7,Mistral-7B,What treatment options are available for Endom...,\n\nTreatment for endometriosis is determined ...,12.714895,Managing Endometriosis involves pain managemen...
8,Mistral-7B,How can I manage my Diabetes?,\n\nType 1 and type 2 diabetes are both chroni...,8.991432,"Medication adherence, blood sugar monitoring."
9,Mistral-7B,How is Dengue Fever diagnosed?,\n\n- Clinical history and physical examinatio...,11.373122,"Lifestyle changes include avoiding alcohol, pr..."


In [19]:
df = df_mistral.append(df_llama, ignore_index=True)

<ipython-input-19-6d02ffdcd12f>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_mistral.append(df_llama, ignore_index=True)


In [20]:
df

,Model,Questions,Reponses,Temps,Reponses_Attendues
0,Mistral-7B,What are the symptoms of Parkinson's Disease?,\n\nThe most common symptoms of Parkinson’s di...,14.755228,Treatment includes medications (levodopa) and ...
1,Mistral-7B,How is HIV/AIDS diagnosed?,\nThe virus is present in blood and can be det...,12.599415,Treatment includes antiretroviral therapy (ART...
2,Mistral-7B,How can I manage my Dengue Fever?,What homeopathy treatments work to combat mos...,13.020158,Diagnosis involves blood tests to detect the v...
3,Mistral-7B,Can Glaucoma be prevented?,"\n\nYes, if it is detected early and treated p...",12.742887,Preventing Glaucoma involves regular eye exams...
4,Mistral-7B,Can Endometriosis be prevented?,\n\nCan you prevent endometriosis? Yes and No....,8.878620,Endometriosis is caused by the abnormal growth...
5,Mistral-7B,How can I manage my Multiple Sclerosis?,"\n\nI’m a newcomer to MS, only being diagnosed...",12.499902,"Parkinson's Disease symptoms include tremors, ..."
6,Mistral-7B,How is Cancer diagnosed?,\nA combination of several tests are used to d...,12.056664,Symptoms vary depending on the type and stage ...
7,Mistral-7B,What treatment options are available for Endom...,\n\nTreatment for endometriosis is determined ...,12.714895,Managing Endometriosis involves pain managemen...
8,Mistral-7B,How can I manage my Diabetes?,\n\nType 1 and type 2 diabetes are both chroni...,8.991432,"Medication adherence, blood sugar monitoring."
9,Mistral-7B,How is Dengue Fever diagnosed?,\n\n- Clinical history and physical examinatio...,11.373122,"Lifestyle changes include avoiding alcohol, pr..."


In [22]:
df.to_csv('llamaMistralResults.csv')

In [3]:
df = pd.read_csv("../data/llamaMistralResults.csv")
df = df.loc[df['Model']=='Mistral-7B']
df

,Unnamed: 0,Model,Questions,Reponses,Temps,Reponses_Attendues
0,0,Mistral-7B,What are the symptoms of Parkinson's Disease?,\n\nThe most common symptoms of Parkinson’s di...,14.755228,Treatment includes medications (levodopa) and ...
1,1,Mistral-7B,How is HIV/AIDS diagnosed?,\nThe virus is present in blood and can be det...,12.599415,Treatment includes antiretroviral therapy (ART...
2,2,Mistral-7B,How can I manage my Dengue Fever?,What homeopathy treatments work to combat mos...,13.020158,Diagnosis involves blood tests to detect the v...
3,3,Mistral-7B,Can Glaucoma be prevented?,"\n\nYes, if it is detected early and treated p...",12.742887,Preventing Glaucoma involves regular eye exams...
4,4,Mistral-7B,Can Endometriosis be prevented?,\n\nCan you prevent endometriosis? Yes and No....,8.878620,Endometriosis is caused by the abnormal growth...
5,5,Mistral-7B,How can I manage my Multiple Sclerosis?,"\n\nI’m a newcomer to MS, only being diagnosed...",12.499902,"Parkinson's Disease symptoms include tremors, ..."
6,6,Mistral-7B,How is Cancer diagnosed?,\nA combination of several tests are used to d...,12.056664,Symptoms vary depending on the type and stage ...
7,7,Mistral-7B,What treatment options are available for Endom...,\n\nTreatment for endometriosis is determined ...,12.714895,Managing Endometriosis involves pain managemen...
8,8,Mistral-7B,How can I manage my Diabetes?,\n\nType 1 and type 2 diabetes are both chroni...,8.991432,"Medication adherence, blood sugar monitoring."
9,9,Mistral-7B,How is Dengue Fever diagnosed?,\n\n- Clinical history and physical examinatio...,11.373122,"Lifestyle changes include avoiding alcohol, pr..."


In [4]:
df['Temps'].mean()

11.963232398033142

In [5]:
print(df.iloc[0])

Unnamed: 0                                                            0
Model                                                        Mistral-7B
Questions                 What are the symptoms of Parkinson's Disease?
Reponses              \n\nThe most common symptoms of Parkinson’s di...
Temps                                                         14.755228
Reponses_Attendues    Treatment includes medications (levodopa) and ...
Name: 0, dtype: object


Le modele Llama-2-7B-AWQ n'est pas concis dans réponses. En effet, il répond en répétant la question qui lui posée(toutefois ceci peut être réglé avec du prétrainement). Le contenu en soi des questions est correct et conforme aux réponses attendues. Cependant, le modèle présente des comportement atypiques en se reposant à lui-même des questions auxquelles il répond. Ses réponses ne sont pas toutes complètes, mais ceci peut être au nombre de tokens qui lui a été demandé de générer.

Le modèle Mistral-7B est largement plus performant que le modele Llama-2-7B. Ce qui est compréhensible, car Mistral est pré-entrainé avec Llama-2. Il répond de maniere précise sans comportements atypiques, comme se reposer des questions à lui même ou ne pas terminer sa réponse. Il a des temps de réponse en moyenne de 11 secondes.